## 네이버 뉴스 페이지에 있는 여러 탭을 크롤링하기
### 탭마다 구조가 다르므로 함수로 처리함

In [338]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

In [104]:
# url을 request하고 html 코드를 받아서 parsing하는 함수
def create_soup(url):
    res = req.get(url, headers = headers)
    soup = bs(res.text, 'html.parser')
    return soup

### 서로 다른 기능을 가진 탭별 크롤링 함수 생성 

### 1. 헤드라인 뉴스 : scrape_headline_news()
'정치', '경제', '사회', '생활/문화', 'IT/과학', '세계'는 구조가 동일함

In [224]:
# '정치', '경제', '사회', '생활/문화', 'IT/과학', '세계'는 구조가 동일함
def scrape_headline_news(sid):
    url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=' + str(sid)
    soup = create_soup(url)
    
    atags = soup.select('ul.sh_list > li > div.sh_text > a')
    contents = soup.select('ul.sh_list > li > div.sh_text > div.sh_text_lede')

    for i in range(len(atags)):        
        title = atags[i].text
        link = atags[i].get('href')
        content = contents[i].text
        print('{}. {}'.format(i+1, title))
        print(link)
        print(content)
        print()

### 2. 랭킹 : scrape_ranking()

In [399]:
def scrape_ranking(papername):
    
    url = 'https://news.naver.com/main/ranking/popularDay.naver?mid=etc&sid1=111'
    soup = create_soup(url)
    
    soup_all = soup.find_all(class_ = re.compile('^_officeCard _officeCard'))
    break_flg = False
    for page in soup_all:
        newspapers = page.select('div.rankingnews_box')
        for newspaper in newspapers:
            atags = newspaper.select('a > strong')
            for atag in atags:
                if atag.text == papername:
                    print(atag.text)
                    get_ranking_news(newspaper)
                    
                    break_flg = True  # 검색대상 언론사를 찾으면 작업 후 빠져 나간다.
                    break
            if break_flg:
                break
        if break_flg:
            break

def get_ranking_news(newspaper):
    articles = newspaper.select("a.list_title")

    for i in range(len(articles)):
        title = articles[i].text
        link = articles[i].get('href')
        print('{}. {}'.format(i+1, title))
        print(link)
        print()

In [402]:
while True:
    #sid = input('검색하고자 하는 뉴스의 분야를 숫자로 입력하세요:\n언론사별:0 정치:1 경제:2 사회:3 생활/문화:4 세계:5 IT/과학:6 랭킹:7 신문보기:8 오피니언:9 팩트체크:10 종료:999\n')
    sid = '7'
    if not sid.isdigit():
        continue
    else:
        sid = int(sid)
        
    if sid == 999:
        break
    elif 0 < sid and sid < 7:
        scrape_headline_news(sid + 99)  # 정치: 100, 경제: 101 ...
    elif sid == 7:
        newspaper = input('언론사명을 입력하세요(예, 한국일보):')
        scrape_ranking(newspaper)
    else:
        print('7부터 10까지')
    
    break

언론사명을 입력하세요(예, 한국일보):경향신문
경향신문
1. 박주민 “원희룡 장관, 부디 사람이 먼저 되시라”
https://n.news.naver.com/article/032/0003224055?ntype=RANKING

2. 연예계 대표 미스터리 김성재 사망 사건, 당시 사건일지부터 단독 인터뷰까지[옛날잡지]
https://n.news.naver.com/article/032/0003224070?ntype=RANKING

3. 문재인 전 대통령 “5·18은 민주주의 뿌리”…퇴임 후 첫 참배
https://n.news.naver.com/article/032/0003224072?ntype=RANKING

4. 간호협회 “오늘부터 대리처방·수술 거부···면허증 반납하겠다”
https://n.news.naver.com/article/032/0003224033?ntype=RANKING

5. ‘음주단속 교통경찰관’이 음주운전 하다 사고
https://n.news.naver.com/article/032/0003224068?ntype=RANKING

